In [1]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 21.98 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (13996/13996), done.


In [2]:
# appending some data and giving key
import csv
data = {}
with open('/content/facial_expressions/data/legend.csv') as d:
  reader  = csv.reader(d)
  next(reader)
  for row in reader:
    key = row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key] = [row[1]]


In [3]:
# making directory in which trainig and testing data will be sub folders

emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [4]:
import os
os.mkdir("master_data")
os.mkdir("master_data/training")
os.mkdir("master_data/testing")

In [5]:
for emotion in emotion_list:
  os.mkdir(f"master_data/training/{emotion}")
  os.mkdir(f"master_data/testing/{emotion}")

In [6]:
from shutil import copyfile
split_size = 0.8
for emotion, images in data.items():
  train_size = int(split_size * len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]

  for image in train_images:
    source = os.path.join("/content/facial_expressions/images", image)
    dest = os.path.join("master_data/training", emotion, image)
    copyfile(source, dest)

    for image in test_images:
      source = os.path.join("/content/facial_expressions/images", image)
      dest = os.path.join("master_data/testing", emotion, image)
      copyfile(source, dest)

#dependenies

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

#model building

In [18]:
model  = tf.keras.models.Sequential([
    Conv2D(16, (3,3), activation="relu", input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024, activation="relu"),
    Dense(8, activation="softmax")
])
model.compile(optimizer=Adam(learning_rate=0.01), loss="categorical_crossentropy", metrics=["acc"])
model.summary()



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 98, 98, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 49, 49, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 47, 47, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 21, 21, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 6400)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1024)                │       6,554,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 8)                   │           8,200 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,586,408 (25.13 MB)

 Trainable params: 6,586,408 (25.13 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
train_dir = '/content/master_data/training'
test_dir = '/content/master_data/testing'


train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory( train_dir,
                                                     target_size=(100,100),
                                                     class_mode='categorical',
                                                     batch_size=12)

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory( test_dir,
                                                     target_size=(100,100),
                                                     class_mode='categorical',
                                                     batch_size=128)




Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [20]:
es = EarlyStopping(monitor='val_acc', patience=2, min_delta=0.01)

In [21]:
model.fit(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=test_generator,
                    callbacks=[es])

Epoch 1/10
912/912 ━━━━━━━━━━━━━━━━━━━━ 182s 198ms/step - acc: 0.4811 - loss: 1.4481 - val_acc: 0.5011 - val_loss: 1.0584
Epoch 2/10
912/912 ━━━━━━━━━━━━━━━━━━━━ 190s 185ms/step - acc: 0.4974 - loss: 1.0548 - val_acc: 0.5011 - val_loss: 1.0459
Epoch 3/10
912/912 ━━━━━━━━━━━━━━━━━━━━ 174s 191ms/step - acc: 0.4906 - loss: 1.0390 - val_acc: 0.4158 - val_loss: 1.0543
